In [ ]:
import glob

In [ ]:
folders = glob.glob("C:/Users/artem/Downloads/Telegram Desktop/ChatExport*")

In [ ]:
files = []
for folder in folders:
    files += glob.glob(f"{folder}/*.html")

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
messages_all = []
for file in files:
    print(file)
    with open(file, encoding='utf-8') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        messages = soup.body.find_all(attrs = {'class':'message'})
        messages= [message.text.split('\n\n')[-2].lower() for message in messages]
    messages_all+=messages
messages_all = list(filter(len, messages_all))

In [ ]:
from nltk.tokenize import sent_tokenize

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(messages_all, columns=['text'])
df = df.drop_duplicates()

In [ ]:
target = 'question'
target_list = []
for message in df['text'].values:
    if 'посовет' in message:
        target_list.append(1)
    else:
        target_list.append(0)   

In [ ]:
df[target] = target_list

In [ ]:
df.shape

In [ ]:
df['question'].sum()

In [ ]:
# len(messages_all)

In [ ]:
# messages_all

In [ ]:
# sent_tokenize(text, language="russian")

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [ ]:
def tokenize(text):
    stop_words = ['за', 'там', 'так', 'то', 'все', 'если', '-',
       ':', 'но', 'как', 'по', 'у', 'есть', 'это', 'я', 'с', 'а', 'не', ')', 'и', '.', 'в', ',']
    tokens = word_tokenize(text, language="russian")
    tokens = list(filter(lambda x:'посовет' not in x, tokens))
    tokens = list(filter(lambda x: x not in stop_words, tokens))
    
    return list(set(tokens))

In [ ]:
df['tokens'] = df['text'].apply(tokenize)

In [ ]:
df = df[df['tokens'].apply(len)>=3]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.33,random_state=42)

In [ ]:
train_lenght = len(df_train)

In [ ]:
mean_target = df_train['question'].mean()

In [ ]:
from functools import reduce

In [ ]:
from collections import Counter

In [ ]:
for text in df[df['question']==1]['text'].values:
    pass
    # print(text)
    # print()

In [ ]:
# df['question'].sum()

In [ ]:
from tqdm import tqdm

In [ ]:
total_count = Counter()
for tokens in tqdm(df_train['tokens'].values):
    for token in tokens:
        total_count[token]+=1
    

In [ ]:
# total_count

In [ ]:
target_count = Counter()
for tokens in tqdm( df_train[df_train[target]==1]['tokens'].values):
    for token in tokens:
        target_count[token]+=1

In [ ]:
pd.Series(total_count).sort_values().index[-30:]

In [ ]:
pd.Series(target_count).sort_values()[-30:]

In [ ]:
pd.Series(target_count).sort_values().iloc[-50:].index

In [ ]:
# total_count

In [ ]:
# tokens = df_test['tokens'].iloc[0]

In [ ]:
# pd.Series(target_count.values()).value_counts()

In [ ]:
def predict(tokens):
    p = 1.0
    for token in tokens:
        p = p*(target_count[token])/(total_count[token]+0.0001)
    return p

In [ ]:
df_train['score'] = df_train['tokens'].apply(predict)

In [ ]:
df_test['score'] = df_test['tokens'].apply(predict)

In [ ]:
# df_test.sort_values('score').iloc[-100:]

In [ ]:
# df_test.sort_values('score')

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(df_train[target], df_train['score'])

In [ ]:
roc_auc_score(df_test[target], df_test['score'])

In [ ]:
df_test['score'].max()

In [ ]:
vc = df[df[target]==1]['tokens'].apply(len).value_counts()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# vc.sort_index()

In [ ]:
plt.bar(vc.index, vc.values)
plt.show()

In [ ]:
# df['text'].value_counts().iloc[-20:100]

In [ ]:
df_test[df_test['tokens'].apply(len)>3].sort_values('score').iloc[-70:]

In [ ]:
# df_test[df_test['question']==1]

In [ ]:
stat = {}

for key, val in target_count.items():
    if val>=3:
        rate = (val)/(total_count[key]+1)
        stat[key] = rate

In [ ]:
pd.Series(stat).sort_values().iloc[-50:]